In [ ]:
import astropy.units as u
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from astroai.tools.utils import set_wcs

figsize = (10, 10)
histsize = (8, 8)
fs = 16

fov = 2.5
binning = 200
pixelsize = (2 * fov) / binning
point_ref = (binning / 2) + (pixelsize / 2)
radius_pix = 0.2/pixelsize

# random IRF

In [ ]:
import pickle
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile, expandvars

# data
zenith = 'random'  # 'z20'
table = 'cleaner_5sgm.pickle'
path = f'{expandvars("$HOME")}/E4/irf_{zenith}/crab/'
dataset = join(path, table)

# dataset 
if '.pickle' in table:
    with open(dataset,'rb') as f: ds = pickle.load(f)
    infotable = join(path, table.replace('.pickle', '.dat'))
    gammatable = join(path, table.replace('.pickle', '_gammapy.txt'))
elif '.npy' in table:
    ds = np.load(dataset, allow_pickle=True, encoding='latin1', fix_imports=True).flat[0]
    infotable = join(path, table.replace('.npy', '.dat'))
    gammatable = join(path, table.replace('.npy', '_gammapy.txt'))
    

print(f"Classes: {ds.keys()}\n")
print(f"NOISY dataset size: {len(ds['DS1'])}")
print(f"CLEAN dataset size: {len(ds['DS2'])}")

ds['DS1'][0].shape, ds['DS2'][0].shape

In [ ]:
from astroai.tools.utils import split_noisy_dataset

train_noisy, train_clean, test_noisy, test_clean = split_noisy_dataset(ds, split=80, reshape=True, binning=200)

print(f"Train clean: {train_clean.shape}")
print(f"Train noisy: {train_noisy.shape}")
print(f"\nTest clean: {test_clean.shape}")
print(f"Test labenoicyls: {test_noisy.shape}")

In [ ]:
infodata = pd.read_csv(infotable, sep=' ', header=0).sort_values(by=['seed'])
infodata.head()

In [ ]:
import tensorflow as tf

savename = 'cleaner_5sgm_filter12_expRAND' 
model = tf.keras.models.load_model(f'../models/cnn_cleaner/{savename}.keras')
model.summary()

In [ ]:
predictions = model.predict(test_noisy)

In [ ]:
loss = model.evaluate(test_noisy, test_clean, verbose=2)

history = np.load(f'../models/cnn_cleaner/{savename}_history.npy', allow_pickle='TRUE').item()
history.keys()

In [ ]:
fig, ax = plt.subplots(figsize=histsize)
ax.tick_params(axis='both', labelsize=fs)
ax.plot(history['loss'], label='training')
ax.plot(history['val_loss'], label = 'validation')
ax.set_xlabel('Epoch', fontsize=fs)
ax.set_ylabel('Loss', fontsize=fs)
ax.set_title('loss function', fontsize=fs*1.5)
ax.set_ylim([0.35,0.7])
ax.grid()
ax.legend(fontsize=fs)

plt.show()
fig.savefig(f'img/talk_rand_loss.png')

In [ ]:
residuals = {'STD': [], 'CNN': []}
for noisy, clean, pred in zip(test_noisy, test_clean, predictions):
    residuals['STD'].append(noisy - clean)
    residuals['CNN'].append(noisy - pred)
    
sum_residual = {'STD': [], 'CNN': []}
for std, cnn in zip(residuals['STD'], residuals['CNN']):
    sum_residual['STD'].append(np.sum(std))
    sum_residual['CNN'].append(np.sum(cnn))

sum_cleaned = {'STD': [], 'CNN': []}
for std, cnn in zip(test_clean, predictions):
    sum_cleaned['STD'].append(np.sum(std))
    sum_cleaned['CNN'].append(np.sum(cnn))
    
sum_original_and_diff = {'NOISY': [], 'DIFF': []}
for orig, std, cnn in zip(test_noisy, residuals['STD'], residuals['CNN']):
    sum_original_and_diff['NOISY'].append(np.sum(orig))
    sum_original_and_diff['DIFF'].append(np.sum(std - cnn))

In [ ]:
colors = ['red', 'blue']

# original hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = sum_original_and_diff['NOISY']
#ax.set_title('simulations (noisy)', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color='green')
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in datatset', fontsize=fs)
ax.set_xlabel('FoV integrated counts', fontsize=fs)
ax.grid() 
fig.savefig(f'img/talk_rand_noisy_counts.png')
plt.show()

# cumulative counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_cleaned['STD'], sum_cleaned['CNN'])
#ax.set_title('simulations vs predictions (clean)', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['simulation', 'prediction'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV integrated counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_rand_clean_counts.png')
plt.show()

In [ ]:
colors = ['red', 'blue']

# residual hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_residual['STD'], sum_residual['CNN'])
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['simulation', 'prediction'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV residual counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_rand_residuals_counts.png')
plt.show()

# diff residual hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = sum_original_and_diff['DIFF']
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color='g')
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV (simulated - predicted) residual counts', fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_rand_residuals_counts_diff.png')
plt.show()

In [ ]:
from astroai.tools.utils import create_circular_mask
from astropy.coordinates import SkyCoord

sum_on_region = {'STD': [], 'CNN': [], 'DIFF': []}

s = len(train_noisy)
for std, cnn in zip(test_clean, predictions):
    s += 1 
    row = infodata[infodata['seed']==s]
    # sky coordinates
    source_deg = {'ra': row['source_ra'].values[0], 'dec': row['source_dec'].values[0]}
    point_deg = {'ra': row['point_ra'].values[0], 'dec': row['point_dec'].values[0]}
    # pixel coordinates
    w = set_wcs(point_ra=row['point_ra'].values[0], point_dec=row['point_dec'].values[0], 
            point_ref=point_ref, pixelsize=pixelsize)
    x, y = w.world_to_pixel(SkyCoord(row['source_ra'].values[0], row['source_dec'].values[0], 
                                                   unit='deg', frame='icrs'))
    # ON counts with STD cleaning
    h, w = std.shape[:2]
    mask = create_circular_mask(h, w, center=(y, x), radius=radius_pix)
    masked_std = std.copy()
    masked_std[~mask] = 0

    # ON counts with CNN cleaning
    h, w = cnn.shape[:2]
    mask = create_circular_mask(h, w, center=(y, x), radius=radius_pix)
    masked_cnn = cnn.copy()
    masked_cnn[~mask] = 0
    
    sum_on_region['STD'].append(np.sum(masked_std))
    sum_on_region['CNN'].append(np.sum(masked_cnn))
    sum_on_region['DIFF'].append(np.sum(masked_std - masked_cnn))

sum_on_region.keys()

In [ ]:
colors = ['red', 'blue'] 

# cumulative counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_on_region['STD'], sum_on_region['CNN']) 
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['simulation', 'prediction'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('excess counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_rand_on_counts.png')
plt.show()

# diff on counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = sum_on_region['DIFF']
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color='g')
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('(simulated - predicted) excess counts', fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_rand_on_counts_diff.png')
plt.show()

# z20 IRF

In [ ]:
import pickle
import numpy as np
import pandas as pd
from os import listdir
from os.path import join, isfile

# data
zenith = 'z20'  # 'random'
table = 'cleaner_5sgm_exp30m_z20.pickle'
path = f'{expandvars("$HOME")}/E4/irf_{zenith}/crab/'
dataset = join(path, table)

# dataset 
if '.pickle' in table:
    with open(dataset,'rb') as f: ds = pickle.load(f)
    infotable = join(path, table.replace('.pickle', '.dat'))
    gammatable = join(path, table.replace('.pickle', '_gammapy.txt'))
elif '.npy' in table:
    ds = np.load(dataset, allow_pickle=True, encoding='latin1', fix_imports=True).flat[0]
    infotable = join(path, table.replace('.npy', '.dat'))
    gammatable = join(path, table.replace('.npy', '_gammapy.txt'))
    
print(f"Classes: {ds.keys()}\n")
print(f"NOISY dataset size: {len(ds['DS1'])}")
print(f"CLEAN dataset size: {len(ds['DS2'])}")

ds['DS1'][0].shape, ds['DS2'][0].shape

In [ ]:
from astroai.tools.utils import split_noisy_dataset

train_noisy, train_clean, test_noisy, test_clean = split_noisy_dataset(ds, split=80, reshape=True, binning=200)

print(f"Train clean: {train_clean.shape}")
print(f"Train noisy: {train_noisy.shape}")
print(f"\nTest clean: {test_clean.shape}")
print(f"Test labenoicyls: {test_noisy.shape}")

In [ ]:
infodata = pd.read_csv(infotable, sep=' ', header=0).sort_values(by=['seed'])
infodata.head()

In [ ]:
import tensorflow as tf

savename = 'cleaner_5sgm_filter12_z20' 
model = tf.keras.models.load_model(f'../models/cnn_cleaner/{savename}.keras')
model.summary()

In [ ]:
predictions = model.predict(test_noisy)

In [ ]:
loss = model.evaluate(test_noisy, test_clean, verbose=2)

history = np.load(f'../models/cnn_cleaner/{savename}_history.npy', allow_pickle='TRUE').item()
history.keys()

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=histsize)
ax.tick_params(axis='both', labelsize=fs)
ax.plot(history['loss'], label='training')
ax.plot(history['val_loss'], label = 'validation')
ax.set_xlabel('Epoch', fontsize=fs)
ax.set_ylabel('Loss', fontsize=fs)
ax.set_title('loss function', fontsize=fs*1.5)
ax.set_ylim([0.40,0.60])
ax.grid()
ax.legend(fontsize=fs)

plt.show()
fig.savefig(f'img/talk_z20_loss.png')

In [ ]:
residuals_z20 = {'STD': [], 'CNN': []}
for noisy, clean, pred in zip(test_noisy, test_clean, predictions):
    residuals_z20['STD'].append(noisy - clean)
    residuals_z20['CNN'].append(noisy - pred)
    
sum_residual_z20 = {'STD': [], 'CNN': []}
for std, cnn in zip(residuals_z20['STD'], residuals_z20['CNN']):
    sum_residual_z20['STD'].append(np.sum(std))
    sum_residual_z20['CNN'].append(np.sum(cnn))

sum_cleaned_z20 = {'STD': [], 'CNN': []}
for std, cnn in zip(test_clean, predictions):
    sum_cleaned_z20['STD'].append(np.sum(std))
    sum_cleaned_z20['CNN'].append(np.sum(cnn))
    
sum_original_and_diff_z20 = {'NOISY': [], 'DIFF': []}
for orig, std, cnn in zip(test_noisy, residuals_z20['STD'], residuals_z20['CNN']):
    sum_original_and_diff_z20['NOISY'].append(np.sum(orig))
    sum_original_and_diff_z20['DIFF'].append(np.sum(std - cnn))

In [ ]:
colors = ['blue', 'red']

# original hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = sum_original_and_diff_z20['NOISY']
#ax.set_title('simulations (noisy)', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color='green')
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in datatset', fontsize=fs)
ax.set_xlabel('FoV integrated counts', fontsize=fs)
ax.grid() 
fig.savefig(f'img/talk_z20_noisy_counts.png')
plt.show()

# cumulative counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_cleaned_z20['STD'], sum_cleaned_z20['CNN'])
#ax.set_title('simulations vs predictions (clean)', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['sim', 'cnn'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV integrated counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_z20_clean_counts.png')
plt.show()

In [ ]:
colors = ['blue', 'red']

# residual hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_residual_z20['STD'], sum_residual_z20['CNN'])
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['sim', 'cnn'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV residual counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_z20_residuals_counts.png')
plt.show()

# residual diff hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = sum_original_and_diff_z20['DIFF']
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color='g')
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV (simulated - predicted) residual counts', fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_z20_residuals_counts_diff.png')
plt.show()

In [ ]:
from astroai.tools.utils import create_circular_mask
from astropy.coordinates import SkyCoord

sum_on_region_z20 = {'STD': [], 'CNN': [], 'DIFF': []}

s = len(train_noisy)
for std, cnn in zip(test_clean, predictions):
    s += 1 
    row = infodata[infodata['seed']==s]
    # sky coordinates
    source_deg = {'ra': row['source_ra'].values[0], 'dec': row['source_dec'].values[0]}
    point_deg = {'ra': row['point_ra'].values[0], 'dec': row['point_dec'].values[0]}
    # pixel coordinates
    w = set_wcs(point_ra=row['point_ra'].values[0], point_dec=row['point_dec'].values[0], 
            point_ref=point_ref, pixelsize=pixelsize)
    x, y = w.world_to_pixel(SkyCoord(row['source_ra'].values[0], row['source_dec'].values[0], 
                                                   unit='deg', frame='icrs'))
    # ON counts with STD cleaning
    h, w = std.shape[:2]
    mask = create_circular_mask(h, w, center=(y, x), radius=radius_pix)
    masked_std = std.copy()
    masked_std[~mask] = 0

    # ON counts with CNN cleaning
    h, w = cnn.shape[:2]
    mask = create_circular_mask(h, w, center=(y, x), radius=radius_pix)
    masked_cnn = cnn.copy()
    masked_cnn[~mask] = 0
    
    sum_on_region_z20['STD'].append(np.sum(masked_std))
    sum_on_region_z20['CNN'].append(np.sum(masked_cnn))
    sum_on_region_z20['DIFF'].append(np.sum(masked_std - masked_cnn))

sum_on_region_z20.keys()

In [ ]:
colors = ['blue', 'red'] 

# cumulative on counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_on_region_z20['STD'], sum_on_region_z20['CNN']) 
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['sim', 'cnn'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('excess counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_z20_on_counts.png')
plt.show()

# diff on counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = sum_on_region_z20['DIFF']
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color='g')
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('(simulated - predicted) excess counts', fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_z20_on_counts_diff.png')
plt.show()

# Comparison

In [ ]:
colors = ['red', 'blue']

# residual diff hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_original_and_diff['DIFF'], sum_original_and_diff_z20['DIFF'][:1000])
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['random IRF', 'z20 IRF'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('FoV (simulated - predicted) residual counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_compare_residuals_counts_diff.png')
plt.show()

# diff on counts hist
fig = plt.figure(figsize=histsize)
ax = fig.add_subplot(111)

data = (sum_on_region['DIFF'], sum_on_region_z20['DIFF'][:1000])
#ax.set_title('residuals', fontsize=fs*1.5)
ax.hist(data, 20, density=False, histtype='step', color=colors, label=['random IRF', 'z20 IRF'])
#ax.tick_params(axis='both', labelsize=fs/2)
ax.set_ylabel('samples in dataset', fontsize=fs)
ax.set_xlabel('(simulated - predicted) excess counts', fontsize=fs)
ax.legend(fontsize=fs)
ax.grid()

fig.savefig(f'img/talk_compare_on_counts_diff.png')
plt.show()